# Welcome to nxneo4j!
#### nxneo4j is a library that enables you to use networkX type of commands to interact with Neo4j. 

Check out the following Mediumn article before you begin:

- https://medium.com/neo4j/nxneo4j-networkx-api-for-neo4j-a-new-chapter-9fc65ddab222
- https://github.com/ybaktir/networkx-neo4j

### _Latest version is 0.0.3_
If not already installed, install the latest version like this:

In [ ]:
# ! pip uninstall -y networkx-neo4j #remove the old installation

In [ ]:
# ! pip install git+https://github.com/ybaktir/networkx-neo4j

In [7]:
import datetime, time
print ('Last run on: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ' ' + repr(time.tzname))

Last run on: 2021-12-10 16:53:22 ('IST', 'IST')


## Connect to Neo4j

Open a free Sandbox session at sandbox.neo4j.com:

https://sandbox.neo4j.com/

Get the connections details like the following

In [8]:
user = 'neo4j'
# password = '84KCr8fxUP5h8nmx8NdIrZIBR5mmrFaIoGoZTkesq5U' # Aura Free password
# uri = 'neo4j+s://01141c89.databases.neo4j.io' # Aura Free URI
password = "unonothing"
uri = "bolt://localhost:7687"

from neo4j import GraphDatabase
import nxneo4j as nx
driver = GraphDatabase.driver(uri=uri,auth=(user,password))
                              #OR "bolt://localhost:7673" for Neo4j Desktop
                              #OR the cloud url
G = nx.Graph(driver)                            

## Add Nodes

In [10]:
#Add a node
G.add_node("Yusuf")

In [ ]:
#Add node with features
G.add_node("Nurgul",gender='F')

In [ ]:
#Add multiple properties at once
G.add_node("Betul",age=4,gender='F')

In [ ]:
#Check nodes
for node in G.nodes():   #Unlike networkX, nxneo4j returns a generator
    print(node)

In [ ]:
#Or simply
list(G.nodes())

In [ ]:
#Get the data associated with each node
list(G.nodes(data=True))

In [ ]:
#number of nodes
len(G)

In [ ]:
#Check a particular node feature
G.nodes['Betul']

In [ ]:
#You can be more specific
G.nodes['Betul']['age']

In [ ]:
G.add_nodes_from([1,2,3,4])

In [ ]:
list(G.nodes())

## Add Edges

In [ ]:
#Add one edge
G.add_edge('Yusuf','Betul')

In [ ]:
#You can change the default connection label like the following
G.relationship_type = 'LOVES'

In [ ]:
G.add_edge('Yusuf','Nurgul')
G.add_edge('Nurgul','Yusuf')

In [ ]:
#You can add properties as well
G.add_edge('Betul','Nurgul',how_much='More than Dad')

In [ ]:
#display the values
list(G.edges(data=True))

In [ ]:
G.relationship_type = 'CONNECTED'

In [ ]:
G.add_edges_from([(1,2),(3,4)])

## Remove Nodes

In [ ]:
G.remove_node('Yusuf')

In [ ]:
list(G.nodes())

## Graph Data Science

There are several builtin graph algorithms in Neo4j. nxneo4j will expand to cover all of them in the future versions. For now, the following networkX algorithms are supported: 
- pagerank
- betweenness_centrality
- closeness_centrality
- label_propagation
- connected_components
- clustering 
- triangles
- shortest_path
- shortest_weighted_path

Let's delete all data and load GOT data:

In [2]:
G.delete_all()
G.load_got()

In [3]:
#You can change the default parameters like the following:
G.identifier_property = 'name'
G.relationship_type = '*'
G.node_label = 'Character'

In [4]:
len(G) #796 nodes

796

In [6]:
nx.draw(G)

## 1. Centrality Algorithms

We’ll start with the famous PageRank algorithm. Let’s find out who the most influential characters in Game of Thrones are:

### Pagerank

We’ll start with the famous PageRank algorithm. Let’s find out who the most influential characters in Game of Thrones are:

In [ ]:
nx.pagerank(G) #RAW OUTPUT

In [ ]:
# the most influential characters
response = nx.pagerank(G)
sorted_pagerank = sorted(response.items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_pagerank[:10]:
    print(character, score)

### Betweenness centrality

We can also run betweenness centrality over the dataset. This algorithm will tell us which nodes are the most 'pivotal' i.e. how many of the shortest paths between pairs of characters must pass through them

In [ ]:
# Betweenness centrality
nx.betweenness_centrality(G) #RAW OUTPUT

In [ ]:
# RANKED OUTPUT
response = nx.betweenness_centrality(G)

sorted_bw = sorted(response.items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_bw[:10]:
    print(character, score)

### Closeness centrality

Closeness centrality tells us on average how many hops away each character is from every other character.

In [ ]:
# Closeness centrality
nx.closeness_centrality(G) #RAW OUTPUT

In [ ]:
# RANKED
response = nx.closeness_centrality(G)

sorted_cc = sorted(response.items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_cc[:10]:
    print(character, score)

## 2. Community Detection Algoritms

### Label Propagation
We can also partition the characters into communities using the label propagation algorithm

In [ ]:
# Label propagation
nx.label_propagation_communities(G) #RAW OUPUT is a generator

In [ ]:
communities = nx.label_propagation_communities(G)
sorted_communities = sorted(communities, key=lambda x: len(x), reverse=True)
for community in sorted_communities[:10]:
    print(list(community)[:10])

Characters are in the same community as those other characters with whom they frequently interact. The idea is that characters have closer ties to those in their community than to those outside.



### Clustering
We can calculate the clustering coefficient for each character. A clustering coefficient of '1' means that all characters that interact with that character also interact with each other:

In [ ]:
# Clustering
nx.clustering(G) #RAW OUTPUT

In [ ]:
response = nx.clustering(G)

biggest_coefficient = sorted(response.items(), key=lambda x: x[1], reverse=True)
for character in biggest_coefficient[:10]:
    print(list(character)[:10])

In [ ]:
list(nx.connected_components(G))

In [ ]:
nx.number_connected_components(G)

In [ ]:
nx.triangles(G) #RAW OUTPUT

## 3. Path Finding Algorithms

Let's find the distance between two characters

In [ ]:
# Shortest path
nx.shortest_path(G, source="Tyrion-Lannister", target="Hodor")

In [ ]:
# Shortest weighted path
nx.shortest_weighted_path(G, source="Tyrion-Lannister", target="Hodor",weight='weight')

In [26]:
import networkx as nx2
G2 = nx2.MultiDiGraph()
G2.add_nodes_from([("Surjit", {"type":"Person"}),("Mahant", {"type":"Person", "age":50}), ("Pune", {"type":"Location"})])
G2.add_edges_from([("Surjit", "Pune", {"label":"lives in"})])


[0]

In [32]:
node_list=[]
edge_list=[]
for node in G2.nodes(data=True):
    print(node)
    node_list.append(node)
    
for edge in G2.edges(data=True):
    print(edge)
    edge_list.append(edge)

G.delete_all()
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

('Surjit', {'type': 'Person'})
('Mahant', {'type': 'Person', 'age': 50})
('Pune', {'type': 'Location'})
('Surjit', 'Pune', {'label': 'lives in'})
